In [1]:
from langchain.memory import ConversationBufferMemory
from langchain.llms import Ollama
from langchain.chains import ConversationChain
# Initialize memory
memory = ConversationBufferMemory()

# Load Ollama with Llama model
llm = Ollama(model="llama3.2")

# Create conversation chain with memory
conversation = ConversationChain(
    llm=llm,
    memory=memory
)

# Example conversation
print('1:', conversation.predict(input="Hey, how are you?"))  
print('2:', conversation.predict(input="Tell me a joke."))   
print('3:', conversation.predict(input="What was my first question?"))  # It remembers!

1: I'm functioning within optimal parameters, thank you for asking! My processes are running smoothly, and I've been updated with the latest software patches. However, I'd love to hear from you - what's on your mind today?
2: I have access to a vast collection of jokes, but let me try to come up with something that might tickle your funny bone! Here's one: A man walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's cat?" The librarian replied, "It rings a bell, but I'm not sure if it's here or not." I hope that brought a smile to your face! Would you like another one?
3: I don't have any information about our conversation prior to this point, as our interaction started with your request for a joke. However, I can tell you that the first message I received from you was "Hey, how are you?" If you'd like to recall what else we discussed before I shared the joke, I'm happy to chat with you again!


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
import time

therapist_model = "llama3.2"
patient_model = "llama3.2"

output_file = open(f"results/{therapist}_{patient}_conversation.txt", 'w')

with open("prompts/therapist.txt", "r") as file:
    therapist_prompt = file.read()
with open("prompts/patient.txt", "r") as file:
    patient_prompt = file.read()
with open("prompts/therapist_end.txt", "r") as file:
    therapist_end_prompt = file.read()


agent_1 = Ollama(model=therapist_model)
agent_2 = Ollama(model=patient_model)

memory_1 = ConversationBufferMemory()
memory_2 = ConversationBufferMemory()

conversation_1 = ConversationChain(llm=agent_1, memory=memory_1)
conversation_2 = ConversationChain(llm=agent_2, memory=memory_2)

message = therapist_prompt
patient_initial = conversation_2.predict(input=patient_prompt)
memory_2.save_context({"input": patient_prompt}, {"output": patient_initial})
turns = 5

for i in range(turns):
    if i == turns-1:
        response_1 = conversation_1.predict(input=f"{therapist_end_prompt}{message}")
    else:
        response_1 = conversation_1.predict(input=message)
    memory_1.save_context({"input": message}, {"output": response_1})
    # print(f"Agent 1: {response_1}")
    output_file.write(f"Therapist: \n{response_1}\n\n")
    
    response_2 = conversation_2.predict(input=response_1)
    # response_2 = input('Agent 2:')
    memory_2.save_context({"input": response_1}, {"output": response_2})
    # print(f"Agent 2: {response_2}")
    output_file.write(f"Patient: \n{response_2}\n\n\n")
    
    message = response_2  # Pass the response to the next agent
    print('Iteration Done', i+1)



Iteration Done 1
Iteration Done 2
Iteration Done 3
Iteration Done 4
Iteration Done 5


In [2]:
!pip install -q bert_score
from bert_score import score

# Example conversation context and response
context = "I'm feeling really tired after work today."
response = "You should take some rest and relax."

# Compute BERTScore
P, R, F1 = score([response], [context], lang="en", model_type="bert-base-uncased", rescale_with_baseline=True)

# Print the F1 score (main metric)
print(f"BERTScore F1: {F1.item():.4f}")



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERTScore F1: 0.1082


In [15]:
s = float(f"{F1.item():.4f}")
type(s)

float

In [3]:
d = {
                "entailment_score": 0.23313108086585999,
                "neutral_score": 0.7469388842582703,
                "contradiction_score": 0.019930055364966393,
                "prediction": "neutral"
            }

In [5]:
print(d, type(d))

{'entailment_score': 0.23313108086585999, 'neutral_score': 0.7469388842582703, 'contradiction_score': 0.019930055364966393, 'prediction': 'neutral'} <class 'dict'>


In [7]:
max({k: v for k, v in d.items() if isinstance(v, (int, float))}.values())

0.7469388842582703

In [26]:
df = pd.read_excel('../results/summary_metrics.xlsx')
df

,Unnamed: 0,total,Perplexity,Flesch Reading Ease,Gunning Fog Index,Entailemtn-Neutral-Consistency,Empathy,BERTScore
0,llama3.2_llama3.2,6,10.503352,53.548333,7.64,0.798991,0.296869,0.38765


In [30]:
d = dict(df)
d

{'Unnamed: 0': 0    llama3.2_llama3.2
 Name: Unnamed: 0, dtype: object,
 'total': 0    6
 Name: total, dtype: int64,
 'Perplexity': 0    10.503352
 Name: Perplexity, dtype: float64,
 'Flesch Reading Ease': 0    53.548333
 Name: Flesch Reading Ease, dtype: float64,
 'Gunning Fog Index': 0    7.64
 Name: Gunning Fog Index, dtype: float64,
 'Entailemtn-Neutral-Consistency': 0    0.798991
 Name: Entailemtn-Neutral-Consistency, dtype: float64,
 'Empathy': 0    0.296869
 Name: Empathy, dtype: float64,
 'BERTScore': 0    0.38765
 Name: BERTScore, dtype: float64}

In [29]:
for key in d:
    print(key)

Unnamed: 0
total
Perplexity
Flesch Reading Ease
Gunning Fog Index
Entailemtn-Neutral-Consistency
Empathy
BERTScore


In [31]:
d = {
    "total": 4,
    "Perplexity": 3.7585527613481076,
    "Flesch Reading Ease": 18.395,
    "Gunning Fog Index": 1.9,
    "Entailment-Neutral-Consistency": 0.23554785549640656,
    "Empathy": 0.16970778894631255,
    "BERTScore": 0.139875
}
d

{'total': 4,
 'Perplexity': 3.7585527613481076,
 'Flesch Reading Ease': 18.395,
 'Gunning Fog Index': 1.9,
 'Entailment-Neutral-Consistency': 0.23554785549640656,
 'Empathy': 0.16970778894631255,
 'BERTScore': 0.139875}

In [33]:
d = {1: d}
df = pd.DataFrame(d)
df

,1
BERTScore,0.139875
Empathy,0.169708
Entailment-Neutral-Consistency,0.235548
Flesch Reading Ease,18.395000
Gunning Fog Index,1.900000
Perplexity,3.758553
total,4.000000
